In [4]:
# 硬投票分类器
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')

from sklearn.metrics import accuracy_score

for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896
VotingClassifier 0.912


In [5]:
# bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,bootstrap=True,n_jobs=-1)
tree_clf = DecisionTreeClassifier()

for clf in (tree_clf, bag_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

DecisionTreeClassifier 0.816
BaggingClassifier 0.904


# AdaBoost

一个跟节点和两个叶子节点组成的树称为树桩stump

AdaBoost具有三个特点：

1. AdaBoost由许多弱学习器组成，弱学习器一般为stump
2. 其中一些stumps具有更多的话语权
3. 每个stump的生成都考虑了前一个stump的错误

## 基本算法

1. 为每一个样本分配一个样本权重，1/样本数量。
2. 创建第一个stump，选择一个属性作为决策变量（基尼系数）。
3. 生成stump的权重
    1. total error 是所有分类错误的样本的样本权重和
    2. 学习器权重$=1/2\log{(1-total error)/total error}$
4. 修改样本的样本权重
    1. 增加错误样本的权重 $sample weight = sample weight* e^{学习器权重}$
    2. 降低的样本权重 $sample weight = sample weight* e^{-学习器权重}$
    3. 对所有的权重归一化，是他们的和为1 
5. 按照样本权重生成新的样本集，大小和原来的集合一样，然后回到第一步创建下一个stump（或者使用weighted gini function 生成下一个stump）

In [6]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1) , n_estimators=300,
    learning_rate=0.2)
for clf in (tree_clf, ada_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

DecisionTreeClassifier 0.84
AdaBoostClassifier 0.912


# 梯度提升

与AdaBoost不同的是，它不是在每个迭代中调整实例权重，而是让新预测器对前一个预测器的残差进行拟合。

最后对所有的预测器的预测结果相加就是对新实例的预测。

In [7]:
# xgboost

import xgboost

xgb_reg = xgboost.XGBClassifier()
for clf in (tree_clf, xgb_reg):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__,accuracy_score(y_pred,y_val))

DecisionTreeClassifier 0.848
[20:34:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier 0.872


# 堆叠法

其基本想法在于不是通过一个简单的函数（比如硬投票）集成所有预测器的预测值，而是训练一个模型去执行这个聚合。

通过留存法，将训练集分成多个子集，第一个子集训练第一层，

第一层的预测器在第二个子集上进行预测，然后将预测值作为特征训练第二层预测器，以此类推可以训练多层堆叠集成的预测器。